In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import json
import math
import csv
import plotly.graph_objects as go

In [ ]:
airdf = pd.read_csv('CSV/runways.csv')
airdf.head()

In [ ]:
airdict = airdf.set_index('id').T.to_dict('list')
print(airdict[3632])

In [ ]:
headdict = {}

for i in airdict:
    headdict[airdict[i][18]] = []
for i in airdict:
    headdict[airdict[i][18]].append(
        [
            airdict[i][0],
            airdict[i][14],
            airdict[i][15],
            airdict[i][16],
            airdict[i][17],
            airdict[i][18],
            airdict[i][19],
            airdict[i][20],
            airdict[i][23],
            airdict[i][24],
        ]
    )


In [ ]:
for i in headdict:
    print(f"{i}: {len(headdict[i])}")

In [ ]:
segs = []
colors = []
my_cmap = plt.get_cmap('jet')

for i in headdict:
    if len(headdict[i]) > 1:
        for j in range(len(headdict[i])):
            x1 = headdict[i][j][7]
            y1 = headdict[i][j][6]
            x2 = headdict[i][j][9]
            y2 = headdict[i][j][8]
            c = headdict[i][j][5]

            colors.append(my_cmap(c))
            segs.append(((x1, y1), (x2, y2)))

fig, ax = plt.subplots()
ax.set_xlim(-180, 180)
ax.set_ylim(-90, 90)

ln_coll = LineCollection(segs, colors=colors, linestyle='solid')

ax.add_collection(ln_coll)
fig.set_size_inches(20, 10)
fig.set_dpi(700)

plt.show()

In [ ]:
with open('JSON/headings.json', 'w') as file:
    file.write(json.dumps(headdict))

In [ ]:
with open("headings.json", "r") as h:
    headdict = json.load(h)

In [ ]:
RADIUS = 0.001
inter = []
touch = []
outside = []
entrylst = []
enddict = {}

In [ ]:
def checkCollision(a, b, c, x, y, radius, entr1, entr2):

    dist = (abs(a * x + b * y + c)) / math.sqrt(a * a + b * b)

    if radius == dist:
        touch.append(1)
        entrylst.append([entr1, entr2])
    elif radius > dist:
        inter.append(1)
        if entr1[0] != entr2[0]:
            entrylst.append([entr1, entr2])
    else:
        outside.append(1)

In [ ]:
for heading in range(1, len(headdict) + 1):
    cur_heading = headdict[str(heading)]
    for i in range(0, len(cur_heading)):
        x = cur_heading[i][4]
        y = cur_heading[i][3]
        entr1 = cur_heading[i]

        for k in range(0, len(cur_heading)):
            a = cur_heading[k][1]
            b = -1
            c = cur_heading[k][2]
            entr2 = cur_heading[k]

            checkCollision(a, b, c, x, y, RADIUS, entr1, entr2)
            
print(f"touch:{len(touch)}\ninter:{len(inter)}\noutside:{len(outside)}")
print(len(entrylst))


In [ ]:
for i in entrylst:
    enddict[i[0][5]] = []

for i in entrylst:
    enddict[i[0][5]].append(i)

In [ ]:
print(enddict[1][1])

In [ ]:
with open("JSON/intersecting.json", "w") as l:
    json.dump(enddict, l)

In [ ]:
with open('JSON/intersecting.json','r') as f:
    headingdict = json.load(f)

In [ ]:
print(headingdict["1"][0][0][0],headingdict["1"][0][0][4])
print(headingdict["1"][0][1][3],headingdict["1"][0][1][4])

In [ ]:
segs2 = []

for heading in range(len(headingdict)):
    for i in headingdict[str(heading+1)]:
        code = i[0][0]
        x1 = i[0][4]
        y1 = i[0][3]
        code2 = i[1][0]
        x2 = i[1][4]
        y2 = i[1][3]

        segs2.append([code,x1,y1,code2,x2,y2])

In [ ]:
print(segs2[1])

In [ ]:
fields = ['airport1','x1', 'y1','airport2', 'x2', 'y2'] 

with open('CSV/intersect.csv', 'w') as f:
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(segs2)

In [ ]:
with open('CSV/intersect-reviewed - intersect.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

print(data[50])

In [ ]:
df = pd.read_csv('CSV/airport-code.csv')
df.head()

In [ ]:
for entry in range(1,len(data)):

    airport1 = data[entry][0]
    airport2 = data[entry][3]

    search1 = df[df['ident'] == airport1]
    search2 = df[df['ident'] == airport2]

    city1 = search1['locale'].item()
    city2 = search2['locale'].item()

    data[entry][6] = city1
    data[entry][7] = city2

print(data[1])

In [ ]:
fields = data[0]
    
rows = data[1:]
  
with open('CSV/intersectingcitys.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(rows)

In [ ]:
df = pd.read_csv("CSV/citytocity.csv")
df.head()

In [ ]:
fig = go.Figure()

source_to_dest = zip(
    df["y1"],
    df["y2"],
    df["x1"],
    df["x2"],
)

## Loop thorugh each flight entry to add line between source and destination
for slat, dlat, slon, dlon in source_to_dest:
    fig.add_trace(
        go.Scattergeo(
            lat=[slat, dlat],
            lon=[slon, dlon],
            mode="lines",
            line=dict(width=1, color="red"),
            # hovertext="none"
        )
    )

cities = df["airport1"].values.tolist()+df["airport2"].values.tolist()
countries = df["city to city"].values.tolist()
scatter_hover_data = [country + " : "+ city for city, country in zip(cities, countries)]

## Loop thorugh each flight entry to plot source and destination as points.
fig.add_trace(
    go.Scattergeo(
        lon=df["x1"].values.tolist() + df["x2"].values.tolist(),
        lat=df["y1"].values.tolist() + df["y2"].values.tolist(),
        hoverinfo="text",
        text=scatter_hover_data,
        mode="markers",
        marker=dict(size=5, color="blue", opacity=0.1),
    )
)

## Update graph layout to improve graph styling.
fig.update_layout(
    title_text="Airport to Airport",
    margin={"t": 0, "b": 0, "l": 0, "r": 0, "pad": 0},
    showlegend=False,
    geo=dict(
        showland=True,
        landcolor="white",
        countrycolor="grey",
        bgcolor="lightgrey",
        projection_type="orthographic",
        showcountries=True,
        projection_rotation={"lon":-90,"lat":15},
    ),
)

fig.show()